In [ ]:
### Data
import xarray as xr
from netCDF4 import Dataset
import numpy as np
from wrf import getvar, interplevel, smooth2d

### Plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import matplotlib.transforms as mtransforms

### Warnings
import warnings
warnings.filterwarnings('ignore')

CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.


In [ ]:
### Parameters
nx = 400
ny = 250
nz = 44
ti = 16
lev = 400
levs = np.arange(300,510,10)

### Read in the data
gfs = Dataset('/p/work1/lloveras/nov2018/30km_files/gfs/wrfin_d01_2018-11-15_12_00_00')
ctl = Dataset('/p/work1/lloveras/nov2018/30km_files/ctl/wrfout_d01_2018-11-13_12_00_00')
full = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_full/wrfout_d01_2018-11-13_12_00_00')
large = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_large/wrfout_d01_2018-11-13_12_00_00')
small = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_small/wrfout_d01_2018-11-13_12_00_00')
box = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_box/wrfout_d01_2018-11-13_12_00_00')
hole = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_hole/wrfout_d01_2018-11-13_12_00_00')

### Latitude and longitude
lats = np.asarray(getvar(ctl,'lat',timeidx=ti))
lons = np.asarray(getvar(ctl,'lon',timeidx=ti))
lons[lons > 0] -= 360

### Pressure
p_gfs = np.asarray(getvar(gfs,'pressure'))
p_ctl = np.asarray(getvar(ctl,'pressure',timeidx=ti))
p_full = np.asarray(getvar(full,'pressure',timeidx=ti))
p_large = np.asarray(getvar(large,'pressure',timeidx=ti))
p_small = np.asarray(getvar(small,'pressure',timeidx=ti))
p_box = np.asarray(getvar(box,'pressure',timeidx=ti))
p_hole = np.asarray(getvar(hole,'pressure',timeidx=ti))

### Height
z_gfs = np.asarray(getvar(gfs,'z'))
zl_gfs = np.asarray(interplevel(z_gfs, p_gfs, lev))

z_ctl = np.asarray(getvar(ctl,'z',timeidx=ti))
zl_ctl = np.asarray(interplevel(z_ctl, p_ctl, lev))

z_full = np.asarray(getvar(full,'z',timeidx=ti))
zl_full = np.asarray(interplevel(z_full, p_full, lev))

z_large = np.asarray(getvar(large,'z',timeidx=ti))
zl_large = np.asarray(interplevel(z_large, p_large, lev))

z_small = np.asarray(getvar(small,'z',timeidx=ti))
zl_small = np.asarray(interplevel(z_small, p_small, lev))

z_box = np.asarray(getvar(box,'z',timeidx=ti))
zl_box = np.asarray(interplevel(z_box, p_box, lev))

z_hole = np.asarray(getvar(hole,'z',timeidx=ti))
zl_hole = np.asarray(interplevel(z_hole, p_hole, lev))

### PV
pv_gfs = np.asarray(getvar(gfs,'pvo'))
pvl_gfs = np.mean(np.asarray(interplevel(pv_gfs, p_gfs, levs)),axis=0)

pv_ctl = np.asarray(getvar(ctl,'pvo',timeidx=ti))
pvl_ctl = np.mean(np.asarray(interplevel(pv_ctl, p_ctl, levs)),axis=0)

pv_full = np.asarray(getvar(full,'pvo',timeidx=ti))
pvl_full = np.mean(np.asarray(interplevel(pv_full, p_full, levs)),axis=0)

pv_large = np.asarray(getvar(large,'pvo',timeidx=ti))
pvl_large = np.mean(np.asarray(interplevel(pv_large, p_large, levs)),axis=0)

pv_small = np.asarray(getvar(small,'pvo',timeidx=ti))
pvl_small = np.mean(np.asarray(interplevel(pv_small, p_small, levs)),axis=0)

pv_box = np.asarray(getvar(box,'pvo',timeidx=ti))
pvl_box = np.mean(np.asarray(interplevel(pv_box, p_box, levs)),axis=0)

pv_hole = np.asarray(getvar(hole,'pvo',timeidx=ti))
pvl_hole = np.mean(np.asarray(interplevel(pv_hole, p_hole, levs)),axis=0)

dpvl_ctl = pvl_ctl - pvl_gfs
dpvl_full = pvl_full - pvl_gfs
dpvl_large = pvl_large - pvl_gfs
dpvl_small = pvl_small - pvl_gfs
dpvl_box = pvl_box - pvl_gfs
dpvl_hole = pvl_hole - pvl_gfs


In [ ]:
### Contour interval
zlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*0.75

### Plot subdomain
min_lat = 27
max_lat = 43
min_lon = -98
max_lon = -82

### Cartopy projection
proj = crs.PlateCarree()

fig, axd = plt.subplot_mosaic([['topleft','topmiddle','topright'],
                               ['bottomleft','bottommiddle','bottomright'],
                               ['cbar','cbar','cbar']],
                              constrained_layout=True, figsize=(7.2,5.4), dpi=200, 
                              gridspec_kw={'width_ratios':[1,1,1],'height_ratios':[1,1,0.05]},
                              per_subplot_kw={'topleft':{'projection':proj},
                                              'topmiddle':{'projection':proj},
                                              'topright':{'projection':proj},
                                              'bottomleft':{'projection':proj},
                                              'bottommiddle':{'projection':proj},
                                              'bottomright':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### TOP LEFT
#############

# Add the gridlines
gls = axd['topleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-95, -90, -85])
gls.ylocator = mticker.FixedLocator([30, 35, 40])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=False
gls.left_labels=True

cs1 = axd['topleft'].contour(lons, lats, smooth2d(zl_gfs,1)/10.,
                  levels=np.arange(0,1500,10),
                  colors='k', transform=crs.PlateCarree())
axd['topleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='k')
cs1 = axd['topleft'].contour(lons, lats, smooth2d(zl_ctl,1)/10.,
                  levels=np.arange(0,1500,10),
                  colors='magenta', transform=crs.PlateCarree())
axd['topleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='magenta')
im1 = axd['topleft'].contourf(lons, lats, dpvl_ctl, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topleft'].set_title('Control')
axd['topleft'].text(0.0, 1.0, '(a)',transform=axd['topleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP MIDDLE
##############

# Add the gridlines
gls = axd['topmiddle'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topmiddle'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-95, -90, -85])
gls.ylocator = mticker.FixedLocator([30, 35, 40])

cs1 = axd['topmiddle'].contour(lons, lats, smooth2d(zl_gfs,1)/10.,
                  levels=np.arange(0,1500,10),
                  colors='k', transform=crs.PlateCarree())
axd['topmiddle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='k')
cs1 = axd['topmiddle'].contour(lons, lats, smooth2d(zl_box,1)/10.,
                  levels=np.arange(0,1500,10),
                  colors='magenta', transform=crs.PlateCarree())
axd['topmiddle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='magenta')
im1 = axd['topmiddle'].contourf(lons, lats, dpvl_box, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topmiddle'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topmiddle'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topmiddle'].set_title('Target')
axd['topmiddle'].text(0.0, 1.0, '(b)',transform=axd['topmiddle'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP RIGHT
##############

# Add the gridlines
gls = axd['topright'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-95, -90, -85])
gls.ylocator = mticker.FixedLocator([30, 35, 40])

cs1 = axd['topright'].contour(lons, lats, smooth2d(zl_gfs,1)/10.,
                  levels=np.arange(0,1500,10),
                  colors='k', transform=crs.PlateCarree())
axd['topright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='k')
cs1 = axd['topright'].contour(lons, lats, smooth2d(zl_small,1)/10.,
                  levels=np.arange(0,1500,10),
                  colors='magenta', transform=crs.PlateCarree())
axd['topright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='magenta')
im1 = axd['topright'].contourf(lons, lats, dpvl_small, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topright'].set_title('Short $\lambda$')
axd['topright'].text(0.0, 1.0, '(c)',transform=axd['topright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM LEFT
##############

# Add the gridlines
gls = axd['bottomleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-95, -90, -85])
gls.ylocator = mticker.FixedLocator([30, 35, 40])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=True

cs1 = axd['bottomleft'].contour(lons, lats, smooth2d(zl_gfs,1)/10.,
                  levels=np.arange(0,1500,10),
                  colors='k', transform=crs.PlateCarree())
axd['bottomleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='k')
cs1 = axd['bottomleft'].contour(lons, lats, smooth2d(zl_full,1)/10.,
                  levels=np.arange(0,1500,10),
                  colors='magenta', transform=crs.PlateCarree())
axd['bottomleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='magenta')
im1 = axd['bottomleft'].contourf(lons, lats, dpvl_full, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomleft'].set_title('Full')
axd['bottomleft'].text(0.0, 1.0, '(d)',transform=axd['bottomleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM MIDDLE
##############

# Add the gridlines
gls = axd['bottommiddle'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottommiddle'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-95, -90, -85])
gls.ylocator = mticker.FixedLocator([30, 35, 40])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=False

cs1 = axd['bottommiddle'].contour(lons, lats, smooth2d(zl_gfs,1)/10.,
                  levels=np.arange(0,1500,10),
                  colors='k', transform=crs.PlateCarree())
axd['bottommiddle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='k')
cs1 = axd['bottommiddle'].contour(lons, lats, smooth2d(zl_hole,1)/10.,
                  levels=np.arange(0,1500,10),
                  colors='magenta', transform=crs.PlateCarree())
axd['bottommiddle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='magenta')
im1 = axd['bottommiddle'].contourf(lons, lats, dpvl_hole, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottommiddle'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottommiddle'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottommiddle'].set_title('Target$^\complement$')
axd['bottommiddle'].text(0.0, 1.0, '(e)',transform=axd['bottommiddle'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM RIGHT
##############

# Add the gridlines
gls = axd['bottomright'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-95, -90, -85])
gls.ylocator = mticker.FixedLocator([30, 35, 40])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.left_labels=False

cs1 = axd['bottomright'].contour(lons, lats, smooth2d(zl_gfs,1)/10.,
                  levels=np.arange(0,1500,10),
                  colors='k', transform=crs.PlateCarree())
axd['bottomright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='k')
cs1 = axd['bottomright'].contour(lons, lats, smooth2d(zl_large,1)/10.,
                  levels=np.arange(0,1500,10),
                  colors='magenta', transform=crs.PlateCarree())
axd['bottomright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='magenta')
im1 = axd['bottomright'].contourf(lons, lats, dpvl_large, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomright'].set_title('Long $\lambda$')
axd['bottomright'].text(0.0, 1.0, '(f)',transform=axd['bottomright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='horizontal', cax=axd['cbar'])
plt.savefig('/p/work1/lloveras/nov2018/figs_pdf/fig08_wrf_30km_48h.pdf',bbox_inches='tight')
plt.show()
